In [ ]:
import zipfile
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50


In [ ]:
# Extract the dataset
zip_file_path = '/content/DOC_CLASSIFICATION.zip'
extract_dir = '/content/DOC_CLASSIFICATION'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [ ]:
# Define paths
train_dir = '/content/DOC_CLASSIFICATION/TRAIN'
val_dir = '/content/DOC_CLASSIFICATION/VAL'
test_dir = '/content/DOC_CLASSIFICATION/TEST'

# Image dimensions
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 10

In [ ]:
# Image data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 126 images belonging to 3 classes.
Found 27 images belonging to 3 classes.


In [ ]:
# Load the ResNet50 model with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(3, activation='softmax')(x)

# Create the complete model
model = Model(inputs=base_model.input, outputs=x)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

94765736/94765736 [==============================] - 4s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                         

In [ ]:
# Train the model
EPOCHS = 20

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)

Epoch 1/20
12/12 [==============================] - 32s 2s/step - loss: 1.3321 - accuracy: 0.3793 - val_loss: 1.0370 - val_accuracy: 0.6000
Epoch 2/20
12/12 [==============================] - 22s 2s/step - loss: 0.8730 - accuracy: 0.6207 - val_loss: 0.7642 - val_accuracy: 0.6500
Epoch 3/20
12/12 [==============================] - 23s 2s/step - loss: 0.7137 - accuracy: 0.6810 - val_loss: 0.7043 - val_accuracy: 0.5500
Epoch 4/20
12/12 [==============================] - 23s 2s/step - loss: 0.6249 - accuracy: 0.7333 - val_loss: 0.5392 - val_accuracy: 0.8500
Epoch 5/20
12/12 [==============================] - 23s 2s/step - loss: 0.5298 - accuracy: 0.8017 - val_loss: 0.7586 - val_accuracy: 0.6500
Epoch 6/20
12/12 [==============================] - 24s 2s/step - loss: 0.5018 - accuracy: 0.8621 - val_loss: 0.7578 - val_accuracy: 0.6500
Epoch 7/20
12/12 [==============================] - 24s 2s/step - loss: 0.4888 - accuracy: 0.8190 - val_loss: 0.6813 - val_accuracy: 0.8500
Epoch 8/20
12/12 [==

In [ ]:
final_val_accuracy = history.history['val_accuracy'][-1]*100
print(f"Final validation accuracy: {final_val_accuracy:.4f}")

Final validation accuracy: 60.0000


In [ ]:
# Directory containing the test images
test_images_dir = '/content/DOC_CLASSIFICATION/TEST'
test_image_paths = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith('.png')]

# Function to preprocess the images
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

# Map predictions to labels
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

# Make predictions
for img_path in test_image_paths:
    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)
    predicted_label = labels[predicted_class[0]]
    print(f'Image: {img_path} -> Predicted label: {predicted_label}')

1/1 [==============================] - 1s 1s/step
Image: /content/DOC_CLASSIFICATION/TEST/doc_000674.png -> Predicted label: scientific_publication
1/1 [==============================] - 0s 187ms/step
Image: /content/DOC_CLASSIFICATION/TEST/doc_000873.png -> Predicted label: email
1/1 [==============================] - 0s 185ms/step
Image: /content/DOC_CLASSIFICATION/TEST/doc_000629.png -> Predicted label: resume
1/1 [==============================] - 0s 182ms/step
Image: /content/DOC_CLASSIFICATION/TEST/doc_000298.png -> Predicted label: scientific_publication
1/1 [==============================] - 0s 174ms/step
Image: /content/DOC_CLASSIFICATION/TEST/doc_000872.png -> Predicted label: email
1/1 [==============================] - 0s 179ms/step
Image: /content/DOC_CLASSIFICATION/TEST/doc_000639.png -> Predicted label: resume
1/1 [==============================] - 0s 175ms/step
Image: /content/DOC_CLASSIFICATION/TEST/doc_000309.png -> Predicted label: resume
1/1 [=======================